### install packages

In [1]:
!pip install --pre --upgrade torch==1.6.0.dev20200411+cu101 torchvision -f https://download.pytorch.org/whl/nightly/cu101/torch_nightly.html
!pip install --pre --upgrade pytorch-ignite 
!pip install --upgrade pynvml fire

Looking in links: https://download.pytorch.org/whl/nightly/cu101/torch_nightly.html
     |████████████████████████████████| 718.1 MB 1.5 kB/s  eta 0:00:01    |█▊                              | 38.9 MB 47.9 MB/s eta 0:00:15     |███                             | 66.7 MB 66.1 MB/s eta 0:00:10     |█████▏                          | 116.0 MB 71.9 MB/s eta 0:00:09     |██████▍                         | 143.7 MB 33.6 MB/s eta 0:00:18     |██████▋                         | 147.2 MB 33.6 MB/s eta 0:00:17     |███████▍                        | 164.6 MB 33.6 MB/s eta 0:00:17     |███████▋                        | 171.6 MB 39.4 MB/s eta 0:00:14     |███████▊                        | 173.4 MB 39.4 MB/s eta 0:00:14     |███████▉                        | 175.2 MB 39.4 MB/s eta 0:00:14     |███████▉                        | 176.9 MB 39.4 MB/s eta 0:00:14     |█████████████▍                  | 301.5 MB 47.7 MB/s eta 0:00:09     |█████████████████               | 381.0 MB 73.2 MB/s eta 0:00:05     |███

In [2]:
#!pip install transformers

In [21]:
import torch
import torch.nn as nn
import random
import time
import math

import pandas as pd
import numpy as np
import collections
from collections import Counter

from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
import sklearn.metrics
from sklearn.metrics import f1_score

from transformers import BertForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import BertTokenizer

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.optim import Optimizer
import torch.nn.functional as F

from torch.cuda.amp import autocast
from torch.cuda.amp import GradScaler


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#seed_val = 17
#random.seed(seed_val)
#np.random.seed(seed_val)
#torch.manual_seed(seed_val)
#torch.cuda.manual_seed_all(seed_val)


print(f'working on {device}, GPU is a {torch.cuda.get_device_name()}')
print(f'using pytorch version {torch.__version__}')

AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx

In [ ]:
#getting the clinical biobert tokenizer
tokenizer = BertTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

In [5]:
#set directories
data_dir = '/floyd/home/data/' 
model_dir = '/floyd/home/models/'

In [7]:
#loads result dataframe or initializes an empty one
try:
    results_df = pd.read_csv(data_dir + 'results.csv', index_col = 0)
except:
    results_df = pd.DataFrame(columns = ['experiment description', 'num samples', 'weighting', 'f1w', 'acc', 'auroc', 'ppv', 'sens', 'batch size'])

results_df.index = list(range(len(results_df)))   #to clean the index 
results_df

,experiment description,num samples,weighting,f1w,acc,auroc,ppv,sens,batch size
0,fresh start floydhub,10000,NaN,0.85842,0.904,0.672198,0.0,NaN,32


### define helper functions

In [8]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return sklearn.metrics.accuracy_score(labels_flat, preds_flat)

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [9]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [10]:
def train_model(model, dataloader_train, dataloader_valid, save_name = None, mixed_prec = True, lr = 1e-5, eps = 1e-8, epochs = 3):
  
    #model.to(device)

    optimizer = AdamW(model.parameters(), lr=lr, eps=eps, weight_decay=0.01)
    
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)
    if mixed_prec == True:
        scaler = GradScaler()
  
    for epoch in tqdm(range(1, epochs+1)):
    
        model.train()
    
        loss_train_total = 0

        progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
        for batch in progress_bar:

            model.zero_grad()
        
            batch = tuple(b.to(device) for b in batch)
        
            inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       


            if mixed_prec == True:
                with autocast():
                    outputs = model(**inputs)
            else:
                outputs = model(**inputs)
        
            loss = outputs[0]
            loss_train_total += loss.item()
            
            if mixed_prec == True:
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                loss.backward()
                optimizer.step()

            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            scheduler.step()
        
            progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
        if save_name:
            torch.save(model.state_dict(), f'{model_dir}finetuned_BERT_{save_name}_epoch_{epoch}.model')
        
        tqdm.write(f'\nEpoch {epoch}')
    
        loss_train_avg = loss_train_total/len(dataloader_train)            
        tqdm.write(f'Training loss: {loss_train_avg}')
    
        val_loss, predictions, true_vals = evaluate(dataloader_validation)
        val_f1 = f1_score_func(predictions, true_vals)
        val_acc = accuracy_score_func(predictions, true_vals)
        tqdm.write(f'Validation loss: {val_loss}')
        tqdm.write(f'F1 Score (Weighted): {val_f1}')
        tqdm.write(f'Validation accuracy: {val_acc}')


In [13]:
def train_model_defined_loss(model, dataloader_train, dataloader_valid, save_name = None, mixed_prec = True, lr = 1e-5, eps = 1e-8, epochs = 3):

    optimizer = AdamW(model.parameters(), lr=lr, eps=eps, weight_decay=0.01)
    
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)
    if mixed_prec == True:
        scaler = GradScaler()
  
    for epoch in tqdm(range(1, epochs+1)):
    
        model.train()
    
        loss_train_total = 0

        progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
        for batch in progress_bar:

            model.zero_grad()
        
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            
            
            #batch = tuple(b.to(device) for b in batch)
        
            #inputs = {'input_ids':      batch[0],
            #      'attention_mask': batch[1],
            #      'labels':         batch[2],
            #   }       


            if mixed_prec == True:
                with autocast():
                    outputs = model(input_ids, attention_mask)
            else:
                outputs = model(input_ids, attention_mask)
        
            loss = loss_func(outputs)
            loss_train_total += loss.item()
            
            if mixed_prec == True:
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                loss.backward()
                optimizer.step()

            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            scheduler.step()
        
            progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
        if save_name:
            torch.save(model.state_dict(), f'{model_dir}finetuned_BERT_{save_name}_epoch_{epoch}.model')
        
        tqdm.write(f'\nEpoch {epoch}')
    
        loss_train_avg = loss_train_total/len(dataloader_train)            
        tqdm.write(f'Training loss: {loss_train_avg}')
    
        val_loss, predictions, true_vals = evaluate(dataloader_validation)
        val_f1 = f1_score_func(predictions, true_vals)
        val_acc = accuracy_score_func(predictions, true_vals)
        tqdm.write(f'Validation loss: {val_loss}')
        tqdm.write(f'F1 Score (Weighted): {val_f1}')
        tqdm.write(f'Validation accuracy: {val_acc}')


In [14]:
def get_metrics(dataloader): #(predictions, true_vals, dataloader):

    _, predictions, true_vals = evaluate(dataloader)

    preds = [np.argmax(pred) for pred in predictions]
    preds_flat = np.argmax(preds).flatten()
    true_vals = true_vals.flatten()

    f1_w = sklearn.metrics.f1_score(true_vals, preds, average='weighted')
    f1 = sklearn.metrics.f1_score(true_vals, preds, average=None)
    acc = sklearn.metrics.accuracy_score(true_vals, preds)
    prec = sklearn.metrics.precision_score(true_vals,preds, average=None) 
    rec = sklearn.metrics.recall_score(true_vals,preds, average=None)
    auroc = sklearn.metrics.roc_auc_score(true_vals,predictions[:,1], average=None)
    confusion = sklearn.metrics.confusion_matrix(true_vals, preds)

    tn, fn, fp, tp = confusion[0,0], confusion[0,1], confusion[1,0], confusion[1,1]

    sens = tp/(tp + fn)
    spec = tn/(tn + fp)
    ppv = tp/(tp + fp)
    npv = tn/(tn + fn)

    print ('Metrics Report:')
    print ('---------------')
    print ('weighted f1: ', f1_w)
    print ('AUROC:       ',auroc)
    print ('accuracy:    ', acc)
    print ('precision:   ', prec)
    print ('recall:      ', rec)
    print ('sensitivity: ', sens)
    print ('specificity: ', spec)
    print ('PPV:         ', ppv)
    print ('NPV:         ', npv)
    print ()
    print ('confusion matrix')
    print (confusion)

    results_df.loc[len(results_df)] = [desc,num_samples, weights, f1_w, acc, auroc, ppv, sens, batch_size]

In [15]:
def encode_data(df, text_field):
    encoded_data_train = tokenizer.batch_encode_plus(
        df[df.data_type=='train'][text_field].values, 
        add_special_tokens=True, 
        return_attention_mask=True, 
        pad_to_max_length=True, 
        max_length=256, 
        return_tensors='pt'
        )

    encoded_data_val = tokenizer.batch_encode_plus(
        df[df.data_type=='val'][text_field].values, 
        add_special_tokens=True, 
        return_attention_mask=True, 
        pad_to_max_length=True, 
        max_length=256, 
        return_tensors='pt'
        )


    input_ids_train = encoded_data_train['input_ids']
    attention_masks_train = encoded_data_train['attention_mask']
    labels_train = torch.tensor(df[df.data_type=='train'].label.values)

    input_ids_val = encoded_data_val['input_ids']
    attention_masks_val = encoded_data_val['attention_mask']
    labels_val = torch.tensor(df[df.data_type=='val'].label.values)

    return input_ids_train, attention_masks_train, labels_train, input_ids_val, attention_masks_val, labels_val



def create_dataloaders(input_ids_train, attention_masks_train, labels_train, input_ids_val, attention_masks_val, labels_val, batch_size = 32):
    dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
    dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

    dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

    dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)
  
    return dataloader_train, dataloader_validation

def process_data(data):
    df = data.sample(num_samples)
    df.discharge.value_counts()
    label_dict = {'discharge':0, 'admit':1}; label_dict
    df['label'] = df.discharge.replace(label_dict)
    df['text'] = df['CleanSubjectiveNotes'].map(str) + ', ' + df['pmhx'].map(str)
    df = df[['discharge', 'label', 'text']]
    display(df.head())
    
    return df, label_dict


def split_data(df, test_size=0.1):
    X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=test_size, 
                                                  random_state=17, 
                                                  stratify=df.label.values)
    df['data_type'] = ['not_set']*df.shape[0]
    df.loc[X_train, 'data_type'] = 'train'
    df.loc[X_val, 'data_type'] = 'val'
    display(df.groupby(['discharge', 'label', 'data_type']).count())
    
    return X_train, X_val, y_train, y_val

In [16]:
class Lamb(Optimizer):
    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-6,
                 weight_decay=0.01, adam=False):   # I changed wd default from 0
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))
        defaults = dict(lr=lr, betas=betas, eps=eps,
                        weight_decay=weight_decay)
        self.adam = adam
        super(Lamb, self).__init__(params, defaults)

    def step(self, closure=None):
        """Performs a single optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                if grad.is_sparse:
                    raise RuntimeError('Lamb does not support sparse gradients, consider SparseAdam instad.')

                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state['step'] = 0
                    # Exponential moving average of gradient values
                    state['exp_avg'] = torch.zeros_like(p.data)
                    # Exponential moving average of squared gradient values
                    state['exp_avg_sq'] = torch.zeros_like(p.data)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                state['step'] += 1

                # Decay the first and second moment running average coefficient
                # m_t
                exp_avg.mul_(beta1).add_(1 - beta1, grad)
                # v_t
                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)

                # Paper v3 does not use debiasing.
                bias_correction1 = 1 - beta1 ** state['step']
                bias_correction2 = 1 - beta2 ** state['step']
                # Apply bias to lr to avoid broadcast.
                step_size = group['lr']  * math.sqrt(bias_correction2) / bias_correction1

                weight_norm = p.data.pow(2).sum().sqrt().clamp(0, 10)

                adam_step = exp_avg / exp_avg_sq.sqrt().add(group['eps'])
                if group['weight_decay'] != 0:
                    adam_step.add_(group['weight_decay'], p.data)

                adam_norm = adam_step.pow(2).sum().sqrt()
                if weight_norm == 0 or adam_norm == 0:
                    trust_ratio = 1
                else:
                    trust_ratio = weight_norm / adam_norm
                state['weight_norm'] = weight_norm
                state['adam_norm'] = adam_norm
                state['trust_ratio'] = trust_ratio
                if self.adam:
                    trust_ratio = 1

                p.data.add_(-step_size * trust_ratio, adam_step)

        return loss


In [17]:

ALPHA = 0.8
GAMMA = 2

class FocalLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(FocalLoss, self).__init__()

    def forward(self, inputs, targets, alpha=ALPHA, gamma=GAMMA, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = torch.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        #first compute binary cross-entropy 
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        BCE_EXP = torch.exp(-BCE)
        focal_loss = alpha * (1-BCE_EXP)**gamma * BCE
                       
        return focal_loss
    
class FocalLoss2(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(FocalLoss2, self).__init__()

    def forward(self, inputs, targets, alpha=ALPHA, gamma=GAMMA, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        #inputs = torch.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        #inputs = inputs.view(-1)
        #targets = targets.view(-1)
        
        #first compute binary cross-entropy 
        BCE = F.binary_cross_entropy_with_logits(inputs, targets, reduction='mean')
        BCE_EXP = torch.exp(-BCE)
        focal_loss = alpha * (1-BCE_EXP)**gamma * BCE
                       
        return focal_loss
    
class JJFocalLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(JJFocalLoss, self).__init__()

    def forward(self, inputs, targets, alpha=0.8, gamma=2, smooth=1):
        
        ce_loss = torch.nn.functional.cross_entropy(inputs, targets, reduction='none') # important to add reduction='none' to keep per-batch-item loss
        pt = torch.exp(-ce_loss)
        focal_loss = (alpha * (1-pt)**gamma * ce_loss).mean()
                       
        return focal_loss


# import the dataframe

In [20]:
data = pd.read_csv(data_dir + 'complete_clean_combo_data.csv', index_col = None, low_memory = False)
#data = pd.read_csv('/floyd/home/data/data_1000.csv', index_col = None)

From prior experimentation it is clear that the only good way to do this training is with GPU2 and mixed precision

In [28]:
weights = None
num_samples = 25000
desc = 'fresh start floydhub gpu2'
batch_size = 32

### Exploratory Data Analysis and Preprocessing

In [29]:
df, label_dict = process_data(data)

,discharge,label,text
126074,discharge,0,patient here for antibiotic. patient was seen ...
151781,admit,1,"complains of right sided flank pain,started x ..."
51640,discharge,0,"as per police , patient has been drinking alco..."
147208,discharge,0,"fever onset yesterday evening, , congested cou..."
85479,discharge,0,patient in emergency room for 7/10 constant ri...


### Training/Validation Split

In [30]:
X_train, X_val, y_train, y_val = split_data(df)

text
discharge label data_type       
admit     1     train       4460
                val          496
discharge 0     train      40540
                val         4504

### Loading Tokenizer and Encoding our Data

In [31]:
input_ids_train, attention_masks_train, labels_train, input_ids_val, attention_masks_val, labels_val = encode_data(df, 'text')

In [32]:
dataloader_train, dataloader_validation = create_dataloaders(input_ids_train, attention_masks_train, labels_train, input_ids_val, attention_masks_val, labels_val,batch_size)

In [33]:
model = BertForSequenceClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device);

### model training 1

In [ ]:
train_model(model, dataloader_train, dataloader_validation, 'floydhub restart', epochs = 3)


Epoch 1
Training loss: 0.28692916471990354
Validation loss: 0.2673689314419297
F1 Score (Weighted): 0.8606609340741481
Validation accuracy: 0.9014



Epoch 2
Training loss: 0.2551324662104429
Validation loss: 0.26223887982451993
F1 Score (Weighted): 0.8756679009526386
Validation accuracy: 0.9044


first time retraining on gpu2 (k100) with mixed precision
this took around 30 minutes for 50000 examples

In [ ]:
get_metrics(dataloader_validation)

In [ ]:
results_df.to_csv('results.csv')

### model training 2

In [23]:
weights = torch.tensor([1,1/8])
num_samples = 25000
desc = 'floydhub gpu2 weighted cross ent'
batch_size = 32

In [24]:
loss_func = nn.CrossEntropyLoss(weight = weights)
train_model_defined_loss(model, dataloader_train, dataloader_validation, epochs = 3)

/pytorch/torch/csrc/utils/python_arg_parser.cpp:754: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, Number alpha)



Epoch 1
Training loss: 0.3172296076907334
Validation loss: 0.3079381734132767
F1 Score (Weighted): 0.8540778537611783
Validation accuracy: 0.901



this takes about 3 minutes for 10000 examples

In [25]:
get_metrics(dataloader_validation)

Metrics Report:
---------------
weighted f1:  0.8540778537611783
AUROC:        0.6731465599390128
accuracy:     0.901
precision:    [0.901 0.   ]
recall:       [1. 0.]
sensitivity:  nan
specificity:  0.901
PPV:          0.0
NPV:          1.0

confusion matrix
[[901   0]
 [ 99   0]]


/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in long_scalars


## Notes to self:

- first step - adamw optimizer, standard precision
- second step - adamw optimizer, mixed precision
- third step - adamw optimizer, label smoothing
- 4th step - adamw optimzer, mixup?
- 5th step - try lamb
- 6th step - try weighted loss
- 7th step - better cleaned data
- 8th step - agumentation of minor class